<h2> 1. Filter out death typesfor both samples and attributes </h2>
<h2> 2. Filter out age groups </h2>

In [1]:
#variables used to select data
dtthrdy_list = [1,2] #see tab with DTHHRDY HERE
age_list = [2,3,4,5,6,7] #see AGE HERE

In [2]:
import pandas as pd
import os 
from pathlib import Path
import copy
import math
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn

pd.set_option('display.max_columns', 100)

In [3]:
parent_path = os.getcwd()

In [4]:
raw_data = 'GTEX_raw_data'
os.chdir(raw_data)

In [5]:
os.listdir()

['GTEx_Analysis_v8_Annotations_SampleAttributesDS.csv',
 'GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct',
 'GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.csv',
 'GTEx_Analysis_v8_Annotations_SubjectPhenotypesDD.xlsx',
 '.ipynb_checkpoints',
 'GTEx_Analysis_v8_Annotations_SampleAttributesDD.xlsx']

In [6]:
pheno = pd.read_csv('GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.csv')

In [7]:
display(pheno.head())

SUBJID  SEX    AGE  DTHHRDY
0  GTEX-1117F    2  60-69      4.0
1  GTEX-111CU    1  50-59      0.0
2  GTEX-111FC    1  60-69      1.0
3  GTEX-111VG    1  60-69      3.0
4  GTEX-111YS    1  60-69      0.0

In [8]:
print(pheno.shape)


(980, 4)


In [9]:
b = pheno['DTHHRDY'].value_counts()
display(b.head(10))


0.0    511
2.0    239
4.0    119
3.0     57
1.0     35
Name: DTHHRDY, dtype: int64

In [10]:
#DTHHRDY HERE
#df[df['A'].isin([3, 6])]

#df[(df['a'] > 1) & (df['a'] < 5)]
pheno_a = pheno[(pheno['DTHHRDY'] > 0) & (pheno['DTHHRDY'] < 3)]

display(pheno_a.head())


SUBJID  SEX    AGE  DTHHRDY
2   GTEX-111FC    1  60-69      1.0
6   GTEX-1128S    2  60-69      2.0
8   GTEX-113JC    2  50-59      2.0
9   GTEX-117XS    1  60-69      2.0
12  GTEX-1192W    1  60-69      2.0

In [11]:
print(pheno_a.shape)

(274, 4)


In [12]:
attributes = pd.read_csv('GTEx_Analysis_v8_Annotations_SampleAttributesDS.csv',index_col=0)
num_row_attributes = attributes.shape[0]
print(num_row_attributes)

22951


In [13]:
display(attributes.sample(2))

SMATSSCR SMCENTER  \
SAMPID                                       
GTEX-S4UY-1426-SM-4AD6Y       1.0       C1   
GTEX-ZTPG-0826-SM-5DUVC       0.0       B1   

                                                                  SMPTHNTS  \
SAMPID                                                                       
GTEX-S4UY-1426-SM-4AD6Y  4 aliquots,  ~6x4mm.  Squamous mucosa present ...   
GTEX-ZTPG-0826-SM-5DUVC                                           2 pieces   

                         SMRIN          SMTS                SMTSD SMUBRID  \
SAMPID                                                                      
GTEX-S4UY-1426-SM-4AD6Y    6.8  Cervix Uteri  Cervix - Ectocervix   12249   
GTEX-ZTPG-0826-SM-5DUVC    7.4       Thyroid              Thyroid    2046   

                         SMTSISCH  SMTSPAX  SMNABTCH  \
SAMPID                                                 
GTEX-S4UY-1426-SM-4AD6Y     433.0    654.0  BP-22893   
GTEX-ZTPG-0826-SM-5DUVC     234.0    700.0  BP-43308   

                                                                 SMNABTCHT  \
SAMPID                                                                       
GTEX-S4UY-1426-SM-4AD6Y                 RNA isolation_PAXgene Tissue miRNA   
GTEX-ZTPG-0826-SM-5DUVC  RNA Extraction from Paxgene-derived Lysate Pla...   

                        SMNABTCHD    SMGEBTCH SMGEBTCHD  SMGEBTCHT SMAFRZE  \
SAMPID                                                                       
GTEX-S4UY-1426-SM-4AD6Y    3/6/12  LCSET-3098   5/14/13  TruSeq.v1  RNASEQ   
GTEX-ZTPG-0826-SM-5DUVC    9/9/13  LCSET-4634    2/9/14  TruSeq.v1  RNASEQ   

                        SMGTC  SME2MPRT  SMCHMPRS  SMNTRART  SMNUMGPS  \
SAMPID                                                                  
GTEX-S4UY-1426-SM-4AD6Y   NaN  0.989119  324040.0  0.969142       NaN   
GTEX-ZTPG-0826-SM-5DUVC   NaN  0.988099  227172.0  0.972565       NaN   

                          SMMAPRT  SMEXNCRT  SM550NRM  SMGNSDTC  SMUNMPRT  \
SAMPID                                                                      
GTEX-S4UY-1426-SM-4AD6Y  0.991324  0.788106       NaN   23675.0       1.0   
GTEX-ZTPG-0826-SM-5DUVC  0.991255  0.807984       NaN   22779.0       1.0   

                         SM350NRM  SMRDLGTH  SMMNCPB  SME1MMRT  SMSFLGTH  \
SAMPID                                                                     
GTEX-S4UY-1426-SM-4AD6Y       NaN      76.0      NaN  0.002046     156.0   
GTEX-ZTPG-0826-SM-5DUVC       NaN      76.0      NaN  0.001874     136.0   

                         SMESTLBS      SMMPPD  SMNTERRT  SMRRNANM     SMRDTTL  \
SAMPID                                                                          
GTEX-S4UY-1426-SM-4AD6Y       0.0  72800000.0  0.030516  163671.0  73500000.0   
GTEX-ZTPG-0826-SM-5DUVC       0.0  65100000.0  0.027097  261630.0  65700000.0   

                           SMVQCFL  SMMNCV  SMTRSCPT    SMMPPDPR  SMCGLGTH  \
SAMPID                                                                       
GTEX-S4UY-1426-SM-4AD6Y  4830000.0     NaN   23709.0  36200000.0       NaN   
GTEX-ZTPG-0826-SM-5DUVC  6420000.0     NaN   22802.0  32400000.0       NaN   

                         SMGAPPCT  SMUNPDRD  SMNTRNRT  SMMPUNRT  SMEXPEFF  \
SAMPID                                                                      
GTEX-S4UY-1426-SM-4AD6Y       NaN       0.0  0.181036  0.991324  0.781269   
GTEX-ZTPG-0826-SM-5DUVC       NaN       0.0  0.164582  0.991255  0.800918   

                           SMMPPDUN  SME2MMRT    SME2ANTI   SMALTALG  \
SAMPID                                                                 
GTEX-S4UY-1426-SM-4AD6Y  72800000.0  0.002786  15900000.0  7200000.0   
GTEX-ZTPG-0826-SM-5DUVC  65100000.0  0.002989  14300000.0  6110000.0   

                           SME2SNSE  SMMFLGTH    SME1ANTI    SMSPLTRD  \
SAMPID                                                                  
GTEX-S4UY-1426-SM-4AD6Y  16000000.0     165.0  16000000.0  13000000.0   
GTEX-

In [14]:
important_attributes = attributes[['SMRIN','SMTSISCH','SMTSD','SMGEBTCH']]
print(important_attributes.shape)

(22951, 4)


In [15]:
display(important_attributes.sample())

SMRIN  SMTSISCH SMTSD    SMGEBTCH
SAMPID                                                    
GTEX-QDT8-0926-SM-32PL2    6.1     581.0  Lung  LCSET-1665

In [16]:
important_attributes = important_attributes.assign(SUBJID = '')
display(important_attributes.head())

SMRIN  SMTSISCH                         SMTSD  \
SAMPID                                                                         
GTEX-1117F-0003-SM-58Q7G         NaN    1188.0                   Whole Blood   
GTEX-1117F-0003-SM-5DWSB         NaN    1188.0                   Whole Blood   
GTEX-1117F-0003-SM-6WBT7         NaN    1188.0                   Whole Blood   
GTEX-1117F-0011-R10a-SM-AHZ7F    NaN    1193.0  Brain - Frontal Cortex (BA9)   
GTEX-1117F-0011-R10b-SM-CYKQ8    7.2    1193.0  Brain - Frontal Cortex (BA9)   

                                       SMGEBTCH SUBJID  
SAMPID                                                  
GTEX-1117F-0003-SM-58Q7G             LCSET-4574         
GTEX-1117F-0003-SM-5DWSB       GTEx_OM25_Dec_01         
GTEX-1117F-0003-SM-6WBT7             LCSET-6056         
GTEX-1117F-0011-R10a-SM-AHZ7F               NaN         
GTEX-1117F-0011-R10b-SM-CYKQ8               NaN

In [17]:
print(important_attributes.shape)
sampids = important_attributes.index.tolist()
print(type(sampids))
print(len(sampids))

(22951, 5)
<class 'list'>
22951


In [18]:
for i in range(important_attributes.shape[0]):
    x = sampids[i].split('-')
    xx = ''
    xx = xx+x[0]+'-'+x[1]
    important_attributes.iloc[i,4] = xx
    

    
    
    

In [19]:
display(important_attributes.head())

SMRIN  SMTSISCH                         SMTSD  \
SAMPID                                                                         
GTEX-1117F-0003-SM-58Q7G         NaN    1188.0                   Whole Blood   
GTEX-1117F-0003-SM-5DWSB         NaN    1188.0                   Whole Blood   
GTEX-1117F-0003-SM-6WBT7         NaN    1188.0                   Whole Blood   
GTEX-1117F-0011-R10a-SM-AHZ7F    NaN    1193.0  Brain - Frontal Cortex (BA9)   
GTEX-1117F-0011-R10b-SM-CYKQ8    7.2    1193.0  Brain - Frontal Cortex (BA9)   

                                       SMGEBTCH      SUBJID  
SAMPID                                                       
GTEX-1117F-0003-SM-58Q7G             LCSET-4574  GTEX-1117F  
GTEX-1117F-0003-SM-5DWSB       GTEx_OM25_Dec_01  GTEX-1117F  
GTEX-1117F-0003-SM-6WBT7             LCSET-6056  GTEX-1117F  
GTEX-1117F-0011-R10a-SM-AHZ7F               NaN  GTEX-1117F  
GTEX-1117F-0011-R10b-SM-CYKQ8               NaN  GTEX-1117F

In [20]:
fill_RIN = important_attributes.groupby('SMTSD')['SMRIN'].mean()

In [21]:
rin_key = fill_RIN.index.tolist()
rin_value = fill_RIN.tolist()
#display(fill_RIN)
print(len(rin_key))
print(rin_key[0])
print(rin_value[0])

zip_iterator = zip(rin_key, rin_value)
#a_dictionary = dict(zip_iterator)
rin_dictionary = dict(zip_iterator)
print(rin_dictionary)

55
Adipose - Subcutaneous
6.86775884665793
{'Adipose - Subcutaneous': 6.86775884665793, 'Adipose - Visceral (Omentum)': 7.016548042704627, 'Adrenal Gland': 7.813602941176471, 'Artery - Aorta': 7.0777777777777775, 'Artery - Coronary': 6.947222222222222, 'Artery - Tibial': 7.273567708333334, 'Bladder': 7.076190476190476, 'Brain - Amygdala': 6.5949152542372875, 'Brain - Anterior cingulate cortex (BA24)': 6.68356807511737, 'Brain - Caudate (basal ganglia)': 7.390034364261169, 'Brain - Cerebellar Hemisphere': 7.45703422053232, 'Brain - Cerebellum': 6.851851851851852, 'Brain - Cortex': 6.5883076923076915, 'Brain - Frontal Cortex (BA9)': 7.168646864686468, 'Brain - Hippocampus': 6.76551724137931, 'Brain - Hypothalamus': 6.855084745762712, 'Brain - Nucleus accumbens (basal ganglia)': 7.14043321299639, 'Brain - Putamen (basal ganglia)': 7.0857758620689655, 'Brain - Spinal cord (cervical c-1)': 7.063186813186813, 'Brain - Substantia nigra': 6.578658536585366, 'Breast - Mammary Tissue': 6.7893750

In [22]:
important_attributes['SMRIN'] = important_attributes['SMRIN'].fillna(-1)
display(important_attributes.sample(50))

SMRIN  SMTSISCH  \
SAMPID                                           
K-562-SM-5J1MB                   9.5       NaN   
GTEX-WFG7-0001-SM-3P61S          9.6    -104.0   
GTEX-13X6I-1526-SM-5SIAC         7.2     826.0   
GTEX-1LGRB-0003-SM-AYROG        -1.0     -44.0   
GTEX-1B8L1-0011-R10b-SM-7P8Q4    7.5     710.0   
GTEX-133LE-1226-SM-5IFGC         7.6     174.0   
GTEX-14JG1-0826-SM-5ZZWR         7.3     948.0   
GTEX-W5WG-1326-SM-4LMI9          7.0     244.0   
GTEX-1H11D-1226-SM-9WPQ2         9.1     332.0   
GTEX-11ZTT-0008-SM-5S2TZ        10.0     246.0   
GTEX-RM2N-0008-SM-48FF3         10.0     227.0   
GTEX-WFG8-0126-SM-4LVMH          8.1     186.0   
GTEX-PVOW-1026-SM-2XCF9          6.4     700.0   
GTEX-1F75I-0226-SM-7MKHQ         7.7     237.0   
GTEX-1J1R8-1326-SM-CE6TC         6.9     383.0   
GTEX-17HHY-0011-R5b-SM-79OMZ     6.6     610.0   
GTEX-OIZI-0126-SM-3NB13          6.7     354.0   
GTEX-PLZ4-1626-SM-3P618          7.6     695.0   
GTEX-P4PP-0926-SM-48TBZ          8.0      66.0   
GTEX-X4EP-0004-SM-4PQZX         10.0    1051.0   
GTEX-P44G-0011-R4A-SM-2I3FB      6.6     574.0   
GTEX-1F6IF-0426-SM-7PBZV         5.8     959.0   
GTEX-1NUQO-0726-SM-E9J3K         8.3     495.0   
GTEX-14BIM-0002-SM-5DU5L        -1.0    1210.0   
GTEX-1EMGI-0001-SM-EAZCK         9.9     352.0   
GTEX-13W46-0726-SM-5LU3Z         7.7     544.0   
GTEX-P4PQ-0526-SM-2D449          7.4     156.0   
GTEX-14PJ4-5004-SM-AN42T        -1.0     130.0   
GTEX-ZPIC-0626-SM-57WDY          8.2     136.0   
GTEX-1269C-2626-SM-5EQ5C         7.9     986.0   
GTEX-147GR-0003-SM-5DU4P        -1.0     868.0   
GTEX-N7MT-0011-R3a-SM-2I3GC      6.4     443.0   
GTEX-1RMOY-0626-SM-E9TIE         8.6     227.0   
GTEX-1IL2U-0126-SM-CNPQ5         6.5     489.0   
GTEX-ZVP2-2226-SM-5GU6Q          7.2     175.0   
GTEX-15CHC-2026-SM-6EU1Z         6.1     252.0   
GTEX-RM2N-1726-SM-2TF55          6.4     223.0   
GTEX-139T8-0004-SM-5HJPE        -1.0     482.0   
GTEX-11ZTT-0003-SM-58Q8I        -1.0     -15.0   
GTEX-1H1DG-0011-R8a-SM-CL54Z     7.0     880.0   
GTEX-1R9K4-1926-SM-DPRZW         7.0    1085.0   
GTEX-1IY9M-0126-SM-D3L8S         8.4    1117.0   
GTEX-1B933-1726-SM-731FC         6.4     734.0   
GTEX-14PHW-2426-SM-69LPU         7.6    1282.0   
GTEX-R55G-2126-SM-2TWB4          7.0     185.0   
GTEX-1AX8Y-0002-SM-7J38S        -1.0     -54.0   
GTEX-V1D1-0006-SM-3NMCE          8.1     -38.0   
GTEX-1KANC-1726-SM-E6CIB         8.0     338.0   
GTEX-1GMR3-1426-SM-7PC16         7.7     280.0   
GTEX-Y3IK-2026-SM-4YCDG          7.7     124.0   

                                                                  SMTSD  \
SAMPID                                                                    
K-562-SM-5J1MB                         Cells - Leukemia cell line (CML)   
GTEX-WFG7-0001-SM-3P61S             Cells - EBV-transformed lymphocytes   
GTEX-13X6I-1526-SM-5SIAC                   Adipose - Visceral (Omentum)   
GTEX-1LGRB-0003-SM-AYROG                                    Whole Blood   
GTEX-1B8L1-0011-R10b-SM-7P8Q4              Brain - Frontal Cortex (BA9)   
GTEX-133LE-1226-SM-5IFGC                                          Ovary   
GTEX-14JG1-0826-SM-5ZZWR                       Heart - Atrial Appendage   
GTEX-W5WG-1326-SM-4LMI9                                          Uterus   
GTEX-1H11D-1226-SM-9WPQ2                         Heart - Left Ventricle   
GTEX-11ZTT-0008-SM-5S2TZ                   Cells - Cultured fibroblasts   
GTEX-RM2N-0008-SM-48FF3                    Cells - Cultured fibroblasts   
GTEX-WFG8-0126-SM-4LVMH                                   Adrenal Gland   
GTEX-PVOW-1026-SM-2XCF9                                            Lung   
GTEX-1F75I-0226-SM-7MKHQ                                         Spleen   
GTEX-1J1R8-1326-SM-CE6TC                                         Testis   
GTEX-17HHY-0011-R5b-SM-79OMZ            Brain - Caudate (basal ganglia)   
GTEX-OIZI-0126-SM-3NB13                                          Testis   
GTEX-

In [23]:
for i in range(important_attributes.shape[0]):
    if important_attributes.iloc[i,0] == -1:
        new_value = rin_dictionary[important_attributes.iloc[i,2]]
        important_attributes.iloc[i,0] = new_value 
        
display(important_attributes.sample(50))


SMRIN  SMTSISCH  \
SAMPID                                              
GTEX-13CZU-0005-SM-5LU8K       9.600000     -80.0   
GTEX-XBED-0008-SM-47JWO        9.400000     173.0   
GTEX-12WSN-0726-SM-5GCMS       7.400000     193.0   
GTEX-YEC4-1226-SM-5CVLX        7.600000     209.0   
GTEX-17HG3-2326-SM-79OKH       7.200000    1194.0   
GTEX-XXEK-0626-SM-5S2SZ        8.100000     145.0   
GTEX-1HFI7-1126-SM-A96S5       5.700000     403.0   
GTEX-1GZHY-0226-SM-9WG6A       6.500000    1134.0   
GTEX-145MF-0626-SM-5LUAI       6.300000     934.0   
GTEX-OHPN-0011-R8A-SM-2IZJ9    5.400000     350.0   
GTEX-11EQ9-0008-SM-5Q5DK      10.000000     321.0   
GTEX-13NZA-1026-SM-5MR48       6.300000    1364.0   
GTEX-YJ89-0526-SM-CYKQA        7.200000    1146.0   
GTEX-147JS-0003-SM-7DRO2       8.156270     404.0   
GTEX-132QS-1626-SM-5K7YZ       8.300000    1591.0   
GTEX-11ZUS-2526-SM-59872       6.700000    1426.0   
GTEX-ZDYS-1926-SM-5HL59        6.700000     203.0   
GTEX-WRHU-0003-SM-CLTFY        8.156270     387.0   
GTEX-TMMY-1126-SM-4DXSX        8.200000     429.0   
GTEX-18D9B-0926-SM-7LG4U       7.200000     455.0   
GTEX-14PK6-1726-SM-6871J       7.100000     147.0   
GTEX-1B8KE-5001-SM-AN461       7.296398     127.0   
GTEX-1JN6P-0626-SM-ARZN6       7.000000     951.0   
GTEX-NPJ7-0011-R1a-SM-3GACT    6.700000     650.0   
GTEX-1A3MX-0126-SM-731BR       8.100000    1124.0   
GTEX-11I78-0005-SM-5N9GB       8.800000     -86.0   
GTEX-16Z82-0426-SM-EZ6Y6       6.779845     522.0   
GTEX-1B97I-0326-SM-7DUGB       7.400000     660.0   
GTEX-1H3VY-0001-SM-EYYVY       9.500000     -30.0   
GTEX-14JG1-0002-SM-7DRNT       8.156270     688.0   
GTEX-1MGNQ-1726-SM-DTX8K       8.400000     284.0   
GTEX-16YQH-0004-SM-6UH22       8.156270     792.0   
GTEX-1N2EE-0002-SM-DJWWO       8.156270    1046.0   
GTEX-RUSQ-1826-SM-2TF6E        7.600000     302.0   
GTEX-UTHO-0003-SM-3G3TQ        8.156270     321.0   
GTEX-Y5LM-0002-SM-4YUV1        8.156270    -469.0   
GTEX-13VXU-0006-SM-5N9FF       8.800000     727.0   
GTEX-11PRG-0726-SM-5EGL1       6.100000     849.0   
GTEX-SIU8-0926-SM-2XV6Q        6.100000    1312.0   
GTEX-1GTWX-0011-R9a-SM-ADEH4   7.300000     875.0   
GTEX-1QPFJ-0626-SM-DTX7Y       6.700000     366.0   
GTEX-13N1W-0011-R9b-SM-5MR4R   7.600000     549.0   
GTEX-1AX8Z-0626-SM-731EH       7.700000     337.0   
GTEX-147F3-0002-SM-5DU5B       8.156270     -36.0   
GTEX-1HCU6-0126-SM-CL54K       8.900000    1081.0   
GTEX-1JMQK-1826-SM-CY8I1       5.900000     963.0   
GTEX-1CAMS-0226-SM-79397       9.500000     355.0   
GTEX-1HFI7-0526-SM-A9SMA       7.500000     372.0   
GTEX-1399U-1426-SM-5J1N8       6.200000     205.0   
GTEX-1MCQQ-0626-SM-E9U5Q       8.400000    1275.0   

                                                              SMTSD  \
SAMPID                                                                
GTEX-13CZU-0005-SM-5LU8K                                Whole Blood   
GTEX-XBED-0008-SM-47JWO                Cells - Cultured fibroblasts   
GTEX-12WSN-0726-SM-5GCMS                                    Thyroid   
GTEX-YEC4-1226-SM-5CVLX       Esophagus - Gastroesophageal Junction   
GTEX-17HG3-2326-SM-79OKH                     Adipose - Subcutaneous   
GTEX-XXEK-0626-SM-5S2SZ                                        Lung   
GTEX-1HFI7-1126-SM-A96S5                                     Vagina   
GTEX-1GZHY-0226-SM-9WG6A                             Nerve - Tibial   
GTEX-145MF-0626-SM-5LUAI             Skin - Sun Exposed (Lower leg)   
GTEX-OHPN-0011-R8A-SM-2IZJ9                    Brain - Hypothalamus   
GTEX-11EQ9-0008-SM-5Q5DK               Cells - Cultured fibroblasts   
GTEX-13NZA-1026-SM-5MR48                                    Thyroid   
GTEX-YJ89-0526-SM-CYKQA                           Muscle - Skeletal   
GTEX-147JS-0003-SM-7DRO2                                Whole Blood   
GTEX-132QS-1626-SM-5K7YZ      Esophagus - Gastroesophageal Junction   
GTEX-11ZUS-2526-SM-59872                                      Liver   

In [24]:
SMTSISCH_mean= important_attributes['SMTSISCH'].mean()
print(SMTSISCH_mean)

552.8053421633554


In [25]:
important_attributes['SMTSISCH'] = important_attributes['SMTSISCH'].fillna(-1)
for i in range(important_attributes.shape[0]):
    if important_attributes.iloc[i,1] == -1:
        new_value = SMTSISCH_mean
        important_attributes.iloc[i,1] = new_value 
    
    if important_attributes.iloc[i,1] < 0:
        new_value = 0
        important_attributes.iloc[i,1] = new_value 
        

display(important_attributes.sample(25))

SMRIN     SMTSISCH  \
SAMPID                                                
GTEX-1H11D-1626-SM-9JGHM       6.50000   354.000000   
GTEX-16MTA-1626-SM-6M47N       7.60000   174.000000   
GTEX-R55G-2126-SM-2TC67        7.00000   185.000000   
GTEX-11ZUS-0011-R11b-SM-5BC7B  9.10000  1268.000000   
K-562-SM-4M1XD                 9.50000   552.805342   
GTEX-17HHE-0126-SM-79398       8.40000   111.000000   
GTEX-R45C-0526-SM-2TWC1        6.50000  1068.000000   
GTEX-1IDJF-1426-SM-ACKWK       7.80000   397.000000   
GTEX-12696-1226-SM-5FQSM       7.80000   533.000000   
GTEX-WYBS-0426-SM-3NM9M        7.40000   861.000000   
GTEX-UPK5-0002-SM-3G3SL        8.15627   795.000000   
GTEX-OXRN-1226-SM-2I5EL        6.00000   721.000000   
GTEX-ZTTD-1026-SM-51MRD        7.10000   336.000000   
GTEX-O5YV-1226-SM-EV795        5.60000   275.000000   
K-562-SM-2TO4S                 9.50000   552.805342   
GTEX-XMK1-1826-SM-4B66F        7.90000   228.000000   
GTEX-1399S-0826-SM-5KM23       6.20000    80.000000   
GTEX-QEL4-1226-SM-447A4        7.90000   532.000000   
GTEX-145MG-0011-R10a-SM-5PNWG  7.90000  1070.000000   
GTEX-XOTO-0011-R7B-SM-4B64R    7.70000   503.000000   
GTEX-18D9U-2126-SM-7LG4Y       8.20000   364.000000   
GTEX-UPK5-1526-SM-4JBJA        8.40000  1003.000000   
GTEX-11OF3-3026-SM-5GU52       7.10000   959.000000   
GTEX-1EN7A-0011-R5a-SM-9QEH6   5.90000   862.000000   
GTEX-P78B-0926-SM-2M48P        6.90000   554.000000   

                                                          SMTSD  \
SAMPID                                                            
GTEX-1H11D-1626-SM-9JGHM                                 Testis   
GTEX-16MTA-1626-SM-6M47N                        Colon - Sigmoid   
GTEX-R55G-2126-SM-2TC67                          Nerve - Tibial   
GTEX-11ZUS-0011-R11b-SM-5BC7B     Brain - Cerebellar Hemisphere   
K-562-SM-4M1XD                 Cells - Leukemia cell line (CML)   
GTEX-17HHE-0126-SM-79398                                  Liver   
GTEX-R45C-0526-SM-2TWC1                       Muscle - Skeletal   
GTEX-1IDJF-1426-SM-ACKWK                                Stomach   
GTEX-12696-1226-SM-5FQSM               Heart - Atrial Appendage   
GTEX-WYBS-0426-SM-3NM9M                         Artery - Tibial   
GTEX-UPK5-0002-SM-3G3SL                             Whole Blood   
GTEX-OXRN-1226-SM-2I5EL                         Artery - Tibial   
GTEX-ZTTD-1026-SM-51MRD                 Breast - Mammary Tissue   
GTEX-O5YV-1226-SM-EV795                         Kidney - Cortex   
K-562-SM-2TO4S                 Cells - Leukemia cell line (CML)   
GTEX-XMK1-1826-SM-4B66F        Small Intestine - Terminal Ileum   
GTEX-1399S-0826-SM-5KM23               Heart - Atrial Appendage   
GTEX-QEL4-1226-SM-447A4                                   Liver   
GTEX-145MG-0011-R10a-SM-5PNWG      Brain - Frontal Cortex (BA9)   
GTEX-XOTO-0011-R7B-SM-4B64R     Brain - Putamen (basal ganglia)   
GTEX-18D9U-2126-SM-7LG4Y                     Esophagus - Mucosa   
GTEX-UPK5-1526-SM-4JBJA                          Artery - Aorta   
GTEX-11OF3-3026-SM-5GU52                              Pituitary   
GTEX-1EN7A-0011-R5a-SM-9QEH6    Brain - Caudate (basal ganglia)   
GTEX-P78B-0926-SM-2M48P                                    Lung   

                                              SMGEBTCH      SUBJID  
SAMPID                                                              
GTEX-1H11D-1626-SM-9JGHM                    LCSET-6805  GTEX-1H11D  
GTEX-16MTA-1626-SM-6M47N                    LCSET-5861  GTEX-16MTA  
GTEX-R55G-2126-SM-2TC67                     LCSET-1477   GTEX-R55G  
GTEX-11ZUS-0011-R11b-SM-5BC7B               LCSET-4455  GTEX-11ZUS  
K-562-SM-4M1XD                              LCSET-3633       K-562  
GTEX-17HHE-0126-SM-79398         LCSET-6376,LCSET-6778  GTEX-17HHE  
GTEX-R45C-0526-SM-2TWC1         GTEx_Affy_Batch7_Apr12   GTEX-R45C  
GTEX-1IDJF-1426-SM-ACKWK                    LCSET-7848  GTEX-1IDJF  
GTEX-12696-1226-SM-5FQSM                    LCS

In [26]:
display(pheno_a.head(20))

SUBJID  SEX    AGE  DTHHRDY
2   GTEX-111FC    1  60-69      1.0
6   GTEX-1128S    2  60-69      2.0
8   GTEX-113JC    2  50-59      2.0
9   GTEX-117XS    1  60-69      2.0
12  GTEX-1192W    1  60-69      2.0
14  GTEX-11DXW    1  40-49      2.0
16  GTEX-11DXY    1  60-69      2.0
18  GTEX-11DYG    1  60-69      2.0
22  GTEX-11EMC    2  60-69      2.0
24  GTEX-11EQ9    1  30-39      2.0
25  GTEX-11GS4    1  60-69      2.0
26  GTEX-11GSO    1  60-69      2.0
27  GTEX-11GSP    2  60-69      2.0
28  GTEX-11H98    1  50-59      2.0
33  GTEX-11NUK    1  50-59      2.0
34  GTEX-11NV4    1  60-69      2.0
35  GTEX-11O72    1  40-49      2.0
36  GTEX-11OC5    1  60-69      2.0
37  GTEX-11OF3    1  60-69      2.0
38  GTEX-11ONC    1  60-69      2.0

In [27]:
new_df  =pd.merge(important_attributes, pheno_a, on=['SUBJID'], how='left')
new_df.index = important_attributes.index

In [28]:
display(new_df.sample(5))


SMRIN  SMTSISCH               SMTSD  \
SAMPID                                                          
GTEX-U3ZH-1026-SM-4DXUZ     9.0     212.0  Esophagus - Mucosa   
GTEX-1GMR2-1526-SM-7MXV6    8.4     200.0  Esophagus - Mucosa   
GTEX-1IDJD-1526-SM-CMKGA    7.3     460.0      Nerve - Tibial   
GTEX-1HCU7-1526-SM-CM2S6    5.9     459.0   Artery - Coronary   
GTEX-15SDE-0826-SM-6M463    7.5     262.0            Pancreas   

                                       SMGEBTCH      SUBJID  SEX  AGE  DTHHRDY  
SAMPID                                                                          
GTEX-U3ZH-1026-SM-4DXUZ   LCSET-3206,LCSET-3797   GTEX-U3ZH  NaN  NaN      NaN  
GTEX-1GMR2-1526-SM-7MXV6             LCSET-6757  GTEX-1GMR2  NaN  NaN      NaN  
GTEX-1IDJD-1526-SM-CMKGA             LCSET-9116  GTEX-1IDJD  NaN  NaN      NaN  
GTEX-1HCU7-1526-SM-CM2S6             LCSET-9115  GTEX-1HCU7  NaN  NaN      NaN  
GTEX-15SDE-0826-SM-6M463             LCSET-5861  GTEX-15SDE  NaN  NaN      NaN

In [29]:
a = new_df['DTHHRDY'].value_counts()
display(a)

2.0    6007
1.0     932
Name: DTHHRDY, dtype: int64

In [30]:
c = new_df['DTHHRDY'].isna().sum()
print(c)

16012


In [31]:
pheno_aa = new_df.dropna(axis=0,how='any')
print(pheno_aa.shape)

(6675, 8)


In [32]:
display(pheno_aa.head(5))

SMRIN  SMTSISCH                           SMTSD  \
SAMPID                                                                        
GTEX-111FC-0001-SM-58Q7U  8.15627     963.0                     Whole Blood   
GTEX-111FC-0001-SM-5DWSO  8.15627     963.0                     Whole Blood   
GTEX-111FC-0001-SM-6WBTJ  8.15627     963.0                     Whole Blood   
GTEX-111FC-0126-SM-5N9DL  7.20000    1036.0  Skin - Sun Exposed (Lower leg)   
GTEX-111FC-0226-SM-5N9B8  7.30000    1040.0          Adipose - Subcutaneous   

                                  SMGEBTCH      SUBJID  SEX    AGE  DTHHRDY  
SAMPID                                                                       
GTEX-111FC-0001-SM-58Q7U        LCSET-4574  GTEX-111FC  1.0  60-69      1.0  
GTEX-111FC-0001-SM-5DWSO  GTEx_OM25_Dec_01  GTEX-111FC  1.0  60-69      1.0  
GTEX-111FC-0001-SM-6WBTJ        LCSET-6056  GTEX-111FC  1.0  60-69      1.0  
GTEX-111FC-0126-SM-5N9DL        LCSET-4904  GTEX-111FC  1.0  60-69      1.0  
GTEX-111FC-0226-SM-5N9B8        LCSET-4904  GTEX-111FC  1.0  60-69      1.0

In [33]:
pheno_aa.describe()

SMRIN     SMTSISCH          SEX      DTHHRDY
count  6675.000000  6675.000000  6675.000000  6675.000000
mean      7.109205   878.741523     1.231311     1.865468
std       0.977786   276.253580     0.421702     0.341248
min       5.100000     0.000000     1.000000     1.000000
25%       6.300000   669.000000     1.000000     2.000000
50%       7.000000   866.000000     1.000000     2.000000
75%       7.900000  1082.000000     1.000000     2.000000
max      10.000000  2029.000000     2.000000     2.000000

In [34]:
for i in range(pheno_aa.shape[0]):

    age = pheno_aa['AGE'].iloc[i]
    #print(age)
    age_int = age[0]
    #print(age_int)

    age_int = int(age_int)
    #print(age_int)
    pheno_aa['AGE'].iloc[i] = age_int 
               
display(pheno_aa.head())

SMRIN  SMTSISCH                           SMTSD  \
SAMPID                                                                        
GTEX-111FC-0001-SM-58Q7U  8.15627     963.0                     Whole Blood   
GTEX-111FC-0001-SM-5DWSO  8.15627     963.0                     Whole Blood   
GTEX-111FC-0001-SM-6WBTJ  8.15627     963.0                     Whole Blood   
GTEX-111FC-0126-SM-5N9DL  7.20000    1036.0  Skin - Sun Exposed (Lower leg)   
GTEX-111FC-0226-SM-5N9B8  7.30000    1040.0          Adipose - Subcutaneous   

                                  SMGEBTCH      SUBJID  SEX AGE  DTHHRDY  
SAMPID                                                                    
GTEX-111FC-0001-SM-58Q7U        LCSET-4574  GTEX-111FC  1.0   6      1.0  
GTEX-111FC-0001-SM-5DWSO  GTEx_OM25_Dec_01  GTEX-111FC  1.0   6      1.0  
GTEX-111FC-0001-SM-6WBTJ        LCSET-6056  GTEX-111FC  1.0   6      1.0  
GTEX-111FC-0126-SM-5N9DL        LCSET-4904  GTEX-111FC  1.0   6      1.0  
GTEX-111FC-0226-SM-5N9B8        LCSET-4904  GTEX-111FC  1.0   6      1.0

In [35]:
#AGE HERE

In [36]:
pheno_aa['AGE'] = pd.to_numeric(pheno_aa["AGE"])

print(pheno_aa.dtypes)


SMRIN       float64
SMTSISCH    float64
SMTSD        object
SMGEBTCH     object
SUBJID       object
SEX         float64
AGE           int64
DTHHRDY     float64
dtype: object


In [37]:
pheno_aa.describe()

SMRIN     SMTSISCH          SEX          AGE      DTHHRDY
count  6675.000000  6675.000000  6675.000000  6675.000000  6675.000000
mean      7.109205   878.741523     1.231311     5.180075     1.865468
std       0.977786   276.253580     0.421702     1.084637     0.341248
min       5.100000     0.000000     1.000000     2.000000     1.000000
25%       6.300000   669.000000     1.000000     5.000000     2.000000
50%       7.000000   866.000000     1.000000     5.000000     2.000000
75%       7.900000  1082.000000     1.000000     6.000000     2.000000
max      10.000000  2029.000000     2.000000     7.000000     2.000000

In [38]:
os.chdir(parent_path)
os.chdir('GTEX_intermediate_data')
print(os.listdir())

['matrix_pcg.csv', '.DS_Store', 'all_sample_attributes.csv', 'dtaa_sample_attributes.csv', '.ipynb_checkpoints', 'pcg_dict_name_to_description.csv', 'pcgSymbols.csv']


In [39]:
pheno_aa.to_csv('all_sample_attributes.csv')

In [40]:
matrix_pcg = pd.read_csv('matrix_pcg.csv',index_col=0)
display(matrix_pcg.head())

GTEX-1117F-0226-SM-5GZZ7  GTEX-1117F-0426-SM-5EGHI  \
Name                                                                     
ENSG00000186092.4                   0.000000                  0.140779   
ENSG00000278566.1                   0.000000                  0.070513   
ENSG00000273547.1                   0.092586                  0.000000   
ENSG00000187634.11                  0.443394                  0.561693   
ENSG00000188976.10                  6.519479                  6.563158   

                    GTEX-1117F-0526-SM-5EGHJ  GTEX-1117F-0626-SM-5N9CS  \
Name                                                                     
ENSG00000186092.4                   0.103451                  0.000000   
ENSG00000278566.1                   0.000000                  0.000000   
ENSG00000273547.1                   0.000000                  0.000000   
ENSG00000187634.11                  0.351854                  3.703765   
ENSG00000188976.10                  6.583008                  6.100137   

                    GTEX-1117F-0726-SM-5GIEN  GTEX-1117F-1326-SM-5EGHH  \
Name                                                                     
ENSG00000186092.4                   0.059812                  0.078733   
ENSG00000278566.1                   0.000000                  0.077011   
ENSG00000273547.1                   0.058497                  0.000000   
ENSG00000187634.11                  3.631104                  1.439357   
ENSG00000188976.10                  4.782409                  5.115616   

                    GTEX-1117F-2426-SM-5EGGH  GTEX-1117F-2526-SM-5GZY6  \
Name                                                                     
ENSG00000186092.4                   0.231801                  0.000000   
ENSG00000278566.1                   0.117948                  0.071323   
ENSG00000273547.1                   0.000000                  0.000000   
ENSG00000187634.11                  4.707083                  1.628074   
ENSG00000188976.10                  6.436628                  6.555509   

                    GTEX-1117F-2826-SM-5GZXL  GTEX-1117F-2926-SM-5GZYI  \
Name                                                                     
ENSG00000186092.4                   0.000000                  0.082839   
ENSG00000278566.1                   0.261350                  0.000000   
ENSG00000273547.1                   0.069908                  0.000000   
ENSG00000187634.11                  0.743213                  0.688583   
ENSG00000188976.10                  6.034304                  6.356672   

                    GTEX-1117F-3026-SM-5GZYU  GTEX-1117F-3226-SM-5N9CT  \
Name                                                                     
ENSG00000186092.4                   0.078200                  0.138028   
ENSG00000278566.1                   0.000000                  0.000000   
ENSG00000273547.1                   0.000000                  0.046435   
ENSG00000187634.11                  1.055890                  0.697418   
ENSG00000188976.10                  6.259272                  5.070389   

                    GTEX-111CU-0126-SM-5GZWZ  GTEX-111CU-0226-SM-5GZXC  \
Name                                                                     
ENSG00000186092.4                   0.098649                  0.062053   
ENSG00000278566.1                   0.049059                  0.118944   
ENSG00000273547.1                   0.096518                  0.000000   
ENSG00000187634.11                  3.691534                  2.387914   
ENSG00000188976.10                  5.265662                  6.252476   

                    GTEX-111CU-0326-SM-5GZXO  GTEX-111CU-0426-SM-5GZY1  \
Name                                                                     
ENSG00000186092.4                   0.062108                  0.048948   
ENSG00000278566.1                   0.060753                  0.047873   
ENSG00000273547.1                   0.060753                  0.047873   
ENSG00000187634.1

In [41]:
print(matrix_pcg.shape)

(18349, 17382)


In [42]:
aa_samples = pheno_aa.index.tolist()
print(len(aa_samples))

6675


In [43]:
matrix_pcg_aa = matrix_pcg.loc[:, matrix_pcg.columns.isin(aa_samples)]

In [44]:
display(matrix_pcg_aa.head())

GTEX-111FC-0126-SM-5N9DL  GTEX-111FC-0226-SM-5N9B8  \
Name                                                                     
ENSG00000186092.4                   0.046980                  0.000000   
ENSG00000278566.1                   0.000000                  0.000000   
ENSG00000273547.1                   0.000000                  0.054598   
ENSG00000187634.11                  1.652142                  1.430820   
ENSG00000188976.10                  6.452035                  5.732541   

                    GTEX-111FC-0326-SM-5GZZ1  GTEX-111FC-0426-SM-5N9CV  \
Name                                                                     
ENSG00000186092.4                   0.137359                  0.087667   
ENSG00000278566.1                   0.091002                  0.000000   
ENSG00000273547.1                   0.134431                  0.000000   
ENSG00000187634.11                  1.067639                  4.214125   
ENSG00000188976.10                  5.137913                  5.920770   

                    GTEX-111FC-0526-SM-5GZZ8  GTEX-111FC-0626-SM-5N9CU  \
Name                                                                     
ENSG00000186092.4                   0.088942                  0.067639   
ENSG00000278566.1                   0.000000                  0.000000   
ENSG00000273547.1                   0.000000                  0.000000   
ENSG00000187634.11                  2.863542                  4.317594   
ENSG00000188976.10                  6.125568                  5.391974   

                    GTEX-111FC-0826-SM-5GZWO  GTEX-111FC-1026-SM-5GZX1  \
Name                                                                     
ENSG00000186092.4                   0.197865                  0.074560   
ENSG00000278566.1                   0.156914                  0.072943   
ENSG00000273547.1                   0.040864                  0.000000   
ENSG00000187634.11                  1.377957                  3.453254   
ENSG00000188976.10                  4.521051                  5.982537   

                    GTEX-111FC-1126-SM-5GZWU  GTEX-111FC-1326-SM-5N9D9  \
Name                                                                     
ENSG00000186092.4                   0.089607                  0.000000   
ENSG00000278566.1                   0.000000                  0.000000   
ENSG00000273547.1                   0.087653                  0.000000   
ENSG00000187634.11                  2.556061                  6.964630   
ENSG00000188976.10                  5.545042                  6.009437   

                    GTEX-111FC-1926-SM-5GZYC  GTEX-111FC-2026-SM-5GZYO  \
Name                                                                     
ENSG00000186092.4                   0.000000                  0.000000   
ENSG00000278566.1                   0.000000                  0.125717   
ENSG00000273547.1                   0.108384                  0.064234   
ENSG00000187634.11                  2.556307                  4.663914   
ENSG00000188976.10                  6.305423                  5.824513   

                    GTEX-111FC-2526-SM-5GZXU  GTEX-111FC-2926-SM-5GZY7  \
Name                                                                     
ENSG00000186092.4                   0.142348                  0.075053   
ENSG00000278566.1                   0.139339                  0.000000   
ENSG00000273547.1                   0.139339                  0.073423   
ENSG00000187634.11                  1.774840                  1.478195   
ENSG00000188976.10                  6.016585                  5.638074   

                    GTEX-111FC-3126-SM-5GZZ2  GTEX-111FC-3326-SM-5GZYV  \
Name                                                                     
ENSG00000186092.4                   0.051553                  0.080126   
ENSG00000278566.1                   0.050425                  0.000000   
ENSG00000273547.1                   0.146264                  0.152768   
ENSG00000187634.1

In [45]:
print(matrix_pcg_aa.shape)

(18349, 5550)


In [46]:
display(new_df.head())

SMRIN  SMTSISCH  \
SAMPID                                              
GTEX-1117F-0003-SM-58Q7G       8.156270    1188.0   
GTEX-1117F-0003-SM-5DWSB       8.156270    1188.0   
GTEX-1117F-0003-SM-6WBT7       8.156270    1188.0   
GTEX-1117F-0011-R10a-SM-AHZ7F  7.168647    1193.0   
GTEX-1117F-0011-R10b-SM-CYKQ8  7.200000    1193.0   

                                                      SMTSD          SMGEBTCH  \
SAMPID                                                                          
GTEX-1117F-0003-SM-58Q7G                        Whole Blood        LCSET-4574   
GTEX-1117F-0003-SM-5DWSB                        Whole Blood  GTEx_OM25_Dec_01   
GTEX-1117F-0003-SM-6WBT7                        Whole Blood        LCSET-6056   
GTEX-1117F-0011-R10a-SM-AHZ7F  Brain - Frontal Cortex (BA9)               NaN   
GTEX-1117F-0011-R10b-SM-CYKQ8  Brain - Frontal Cortex (BA9)               NaN   

                                   SUBJID  SEX  AGE  DTHHRDY  
SAMPID                                                        
GTEX-1117F-0003-SM-58Q7G       GTEX-1117F  NaN  NaN      NaN  
GTEX-1117F-0003-SM-5DWSB       GTEX-1117F  NaN  NaN      NaN  
GTEX-1117F-0003-SM-6WBT7       GTEX-1117F  NaN  NaN      NaN  
GTEX-1117F-0011-R10a-SM-AHZ7F  GTEX-1117F  NaN  NaN      NaN  
GTEX-1117F-0011-R10b-SM-CYKQ8  GTEX-1117F  NaN  NaN      NaN

In [47]:
aa_matrix_samples = matrix_pcg_aa.columns.to_list()
print(len(aa_matrix_samples))

5550


In [48]:
new_df_zero = new_df.loc[new_df.index.isin(aa_matrix_samples),:]

In [49]:
print(new_df_zero.shape)

(5550, 8)


In [50]:
new_df_zero.to_csv('dtaa_sample_attributes.csv')

In [51]:
matrix_pcg_aa.to_csv('matrix_pcg_aa.csv')